## BT-LAB .mpt with loops to csv converter

In [1]:
import pandas as pd
import os

In [2]:
# this function read the files from the directory of the script
# return the files in the directory

def seekfiles():
    files = []
    for file in os.listdir("."):
        if file.endswith(".mpt"): # take all the files that ends with .mpt format and append the file into the list
            files.append(file)
    return files # return list of files with .mpt format 

In [3]:
# This function converts the given content of .mpt BT-LAB file into pandas dataset.
# parameter: filecontents --> the contents of the file from readlines()
# return: dataset --> return the dataset as a pandas dataframe

def convertToDataset(filecontents):
    newContent = []
    for lines in filecontents:
        newContent.append(lines.strip()) # delete the spacing \n from each lines
    headerLine = newContent[1][18:] # take the line number of the header in the .mpt file
    tableContent = newContent[int(headerLine)-1] # saved the column values of the data into a list
    header = tableContent.split('\t') # split the tableContent (headers list) from the tabular splitter.
    data = filecontents[int(headerLine):] # take the raw data starting from the header line + 1
    
    newData = [] # the data that is going to be used in making the pandas dataframe,
    for val in data:
        newData.append(val.rstrip().split('\t')) # append each raw data into newData with deleted tabular separator
    df = pd.DataFrame(newData, columns=header) # create a new pandas dataframe using newData list from before.
    dataset = df.apply(lambda x: x.str.replace(',','.')) # replace all comas in data into dots
    dataset = dataset.astype(float) # set all data from string into float
    return dataset 

In [20]:
# parameter: filecontents --> file content from .readlines
#            dataframe    --> pandas dataframe

def convertLoops(filecontents, dataframe):
    line = None
    
    # find the line where the loop number is located from the filecontent.
    for num, lines in enumerate(filecontents, 1):
        # find loop in the filecontent
        if 'loops' in lines:
            line = num
            
    # take the number of loops string from the .mpt file located in the line variable found above.        
    numLoops = filecontents[line-1][18:] 
    loops = filecontents[line:line+int(numLoops)] # a list of string of the number of loops
    
    print('Number of loops: {}'.format(numLoops))
    
    loopStart = [] # -- X ROW -- starting point of new loop (x row) to loop end (y row) in lists
    loopEnd = [] # -- Y ROW -- end point of loop in lists
    
    for line in loops:
        loopStart.append(line.split()[5]) # take the starting point from the string list
    for line in loops:
        loopEnd.append(line.split()[7]) # take the end point from the string list
        
    dataLoops = []
    for i in range(len(loops)):
        # append from row x to row y specified by the loopStart and loopEnd variable above.
        dataLoops.append(dataframe.iloc[int(loopStart[i]):int(loopEnd[i])+1])
    return dataLoops

In [13]:
# output of seekfiles saved in a list

files = seekfiles()
files

['20220309_(Form02C)EIS_charging_10bis16V_step_01V+(1C+EIS)50cycle_06_GCPL6_C08.mpt',
 '20220309_(Form02C)EIS_charging_10bis16V_step_01V+(1C+EIS)50cycle_16_GCPL6_C08.mpt',
 '20220309_(Form02C)EIS_charging_10bis16V_step_01V+(1C+EIS)50cycle_17_GEIS_C08.mpt']

In [18]:
# convert the files using the defined functions.

def toDF(files):
    datasets = []
    for file in files:
        print('Converting to .csv ... \nFilename: {} to .csv'.format(file))
        f = open(file,'r',encoding='ISO-8859-1')
        content = f.readlines()
        datasets.append(convertLoops(content, convertToDataset(content)))
    return datasets

In [21]:
def write_csv(dataset, files):
    for i, data in enumerate(dataset):
        print('converting {0} to .csv ...'.format(files[i]))
        for j, loops in enumerate(data):
            print('Converting loop {0} to .csv ...'.format(j))
            loops.to_csv('{file}__loop{loopNum}.csv'.format(file = files[i], loopNum=j), index=False)
        print('converted {0} to .csv'.format(files[i]))

In [23]:
# in action:

# put the files into the dfs list.

dfs = toDF(seekfiles())

Converting to .csv ... 
Filename: 20220309_(Form02C)EIS_charging_10bis16V_step_01V+(1C+EIS)50cycle_06_GCPL6_C08.mpt to .csv
Number of loops: 1

Converting to .csv ... 
Filename: 20220309_(Form02C)EIS_charging_10bis16V_step_01V+(1C+EIS)50cycle_16_GCPL6_C08.mpt to .csv
Number of loops: 50

Converting to .csv ... 
Filename: 20220309_(Form02C)EIS_charging_10bis16V_step_01V+(1C+EIS)50cycle_17_GEIS_C08.mpt to .csv
Number of loops: 50



In [33]:
# To access a dataframe from the current folder: dfs[x][y] ----> x = database, y = loop index
# example below: 

dfs[2][0]

,freq/Hz,Re(Z)/Ohm,-Im(Z)/Ohm,|Z|/Ohm,Phase(Z)/deg,time/s,<Ewe>/V,<I>/mA,Cs/µF,Cp/µF,...,Phase(Zwe-ce)/deg,|Zwe-ce|/Ohm,Re(Zwe-ce)/Ohm,-Im(Zwe-ce)/Ohm,P/W,Re(Y)/Ohm-1,Im(Y)/Ohm-1,|Y|/Ohm-1,Phase(Y)/deg,dq/mA.h
0,1.000019e+06,-29.241573,45.741653,54.289673,-122.589840,63461.635038,13.571516,0.000013,3.479367e-03,0.002470,...,-99.941147,22.212055,-3.834612,21.878555,0.0,-0.009921,0.015519,0.018420,122.589840,1.500450e-09
1,6.770060e+05,-13.077118,35.164925,37.517769,-110.399140,63462.151038,13.571962,0.000014,6.685255e-03,0.005873,...,-71.517822,29.954691,9.495926,28.409697,0.0,-0.009290,0.024982,0.026654,110.399140,2.447751e-10
2,4.583302e+05,-3.746337,36.195190,36.388554,-95.909286,63462.536038,13.571782,0.000014,9.593802e-03,0.009492,...,-67.300781,53.585804,20.678413,49.435226,0.0,-0.002829,0.027335,0.027481,95.909286,4.054343e-09
3,3.102917e+05,13.878423,36.036575,38.616646,-68.937286,63462.921038,13.571808,0.000019,1.423333e-02,0.012395,...,-59.494522,74.761055,37.950260,64.412674,0.0,0.009307,0.024165,0.025896,68.937286,-3.478105e-09
4,2.100746e+05,28.782810,18.307480,34.111786,-32.458649,63463.306038,13.571802,0.000016,4.138262e-02,0.011920,...,-53.406418,89.852943,53.564480,72.141510,0.0,0.024736,0.015733,0.029315,32.458649,9.253153e-09
5,1.422209e+05,16.881842,-4.731371,17.532326,15.656269,63463.560038,13.571827,0.000015,-2.365210e-01,-0.017225,...,-55.271717,92.414925,52.647423,75.952400,0.0,0.054921,-0.015392,0.057037,-15.656269,2.627382e-09
6,9.628303e+04,12.319058,-6.298673,13.835913,27.080431,63463.814038,13.571873,0.000015,-2.624347e-01,-0.054388,...,-57.080654,124.368710,67.589157,104.399620,0.0,0.064352,-0.032903,0.072276,-27.080431,5.092516e-09
7,6.518148e+04,6.254584,-7.069839,9.439409,48.501278,63464.199038,13.571795,0.000019,-3.453714e-01,-0.193738,...,-59.472023,161.275300,81.921242,138.919510,0.0,0.070195,-0.079345,0.105939,-48.501278,2.642781e-09
8,4.412856e+04,4.704019,-2.961658,5.558706,32.194588,63464.584038,13.571863,0.000015,-1.217770e+00,-0.345691,...,-60.340061,219.604200,108.671410,190.831160,0.0,0.152237,-0.095849,0.179898,-32.194588,-2.135240e-09
9,2.987678e+04,4.020914,-2.193924,4.580508,28.618126,63464.969038,13.571793,0.000016,-2.428090e+00,-0.557033,...,-58.320076,291.071040,152.862790,247.700040,0.0,0.191645,-0.104567,0.218316,-28.618126,2.168434e-09


In [30]:
# convert each dataframe to csv, taking the files as a parameter for the file names.

write_csv(dfs, files)

converting 20220309_(Form02C)EIS_charging_10bis16V_step_01V+(1C+EIS)50cycle_06_GCPL6_C08.mpt to .csv ...
Converting loop 0 to .csv ...
converted 20220309_(Form02C)EIS_charging_10bis16V_step_01V+(1C+EIS)50cycle_06_GCPL6_C08.mpt to .csv
converting 20220309_(Form02C)EIS_charging_10bis16V_step_01V+(1C+EIS)50cycle_16_GCPL6_C08.mpt to .csv ...
Converting loop 0 to .csv ...
Converting loop 1 to .csv ...
Converting loop 2 to .csv ...
Converting loop 3 to .csv ...
Converting loop 4 to .csv ...
Converting loop 5 to .csv ...
Converting loop 6 to .csv ...
Converting loop 7 to .csv ...
Converting loop 8 to .csv ...
Converting loop 9 to .csv ...
Converting loop 10 to .csv ...
Converting loop 11 to .csv ...
Converting loop 12 to .csv ...
Converting loop 13 to .csv ...
Converting loop 14 to .csv ...
Converting loop 15 to .csv ...
Converting loop 16 to .csv ...
Converting loop 17 to .csv ...
Converting loop 18 to .csv ...
Converting loop 19 to .csv ...
Converting loop 20 to .csv ...
Converting loop 21 

In [35]:
dfs[0][0]

,mode,ox/red,error,control changes,Ns changes,counter inc.,Ns,time/s,dq/mA.h,(Q-Qo)/mA.h,...,Capacitance discharge/µF,<I>/mA,Q discharge/mA.h,Q charge/mA.h,Capacity/mA.h,Efficiency/%,control/V,control/mA,cycle number,Ecell/V
0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,12398.749049,0.000000e+00,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,-3.079619
1,3.0,1.0,0.0,0.0,0.0,0.0,0.0,12399.749049,0.000000e+00,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.920086
2,3.0,1.0,0.0,0.0,0.0,0.0,0.0,12400.749049,0.000000e+00,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.918802
3,3.0,1.0,0.0,0.0,0.0,0.0,0.0,12401.749049,0.000000e+00,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.918201
4,3.0,1.0,0.0,0.0,0.0,0.0,0.0,12402.749049,0.000000e+00,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.916919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,2.0,1.0,0.0,0.0,0.0,0.0,1.0,15981.934406,3.209802e-04,0.013759,...,0.0,0.010092,0.0,0.013759,0.013759,0.0,1.199983,0.0,0.0,1.196971
86,2.0,1.0,0.0,0.0,0.0,0.0,1.0,15981.934606,5.540155e-10,0.013759,...,0.0,0.009972,0.0,0.013759,0.013759,0.0,1.199983,0.0,0.0,1.196651
87,3.0,1.0,0.0,1.0,0.0,0.0,1.0,15981.983006,0.000000e+00,0.013759,...,0.0,0.000000,0.0,0.013759,0.013759,0.0,0.000000,0.0,0.0,-3.078837
88,3.0,1.0,0.0,1.0,0.0,0.0,1.0,16041.885205,0.000000e+00,0.013759,...,0.0,0.000000,0.0,0.013759,0.013759,0.0,0.000000,0.0,0.0,1.106284
